Unleash the Ray

In [ ]:
%load_ext autoreload
%autoreload 2

from dependencies import *
from tuning import *

### Let's start Ray

In [ ]:
ray.init(num_cpus=6, num_gpus=0, include_webui=True)

After initialisation the [Ray Dashboard](https://docs.ray.io/en/master/ray-dashboard.html) is available on the **webui_url** port

## Go Random Go!

#### TODO replace grid_search

Replace grid search with other tune distribution that we can sample from >> [read the docs](https://docs.ray.io/en/latest/tune/api_docs/grid_random.html?highlight=tune.grid#random-distributions-api)

Choose appropriate distriutions for the different parameter types

In [ ]:
ray_tuning_config = {
    'randomforestclassifier__n_estimators': tune.randint(1, 150),
    'randomforestclassifier__criterion': tune.choice(['gini', 'entropy']),
    'randomforestclassifier__max_features': tune.choice(['auto', 'sqrt', 'log2']),
    'randomforestclassifier__bootstrap': tune.choice([True, False]),
    'randomforestclassifier__min_samples_leaf': tune.randint(1,4),
    'randomforestclassifier__min_samples_split': tune.randint(3,6)
}

In [ ]:
analysis = tune.run(
                e2e_simple_training,
                config=ray_tuning_config,
                num_samples=100, # Specify the number of samples to make from (non grid) distributions
                resources_per_trial=dict(cpu=1, gpu=0))

In [ ]:
print("Best config: ", analysis.get_best_config(metric="mean_f1_score"))

In [ ]:
df = analysis.dataframe()
top_n_df = df.nlargest(10, "mean_f1_score")

In [ ]:
plot_some_tune_results(top_n_df)

In [ ]:
%load_ext tensorboard
from tensorboard import notebook 
%tensorboard --logdir "~/ray_results/grid_search"

In [ ]:
ray.shutdown()

exit

So what have we just done?
 - shown how the performance of a model with a single set of hyperparameters affects our score
 - shown that kfold cv can give us a picture of that distribution which is easier to compute that out-of-bootstrap scores and has less leakage
 - so which model to choose? == in this case which data split, which is not what we want, we want to chose a robust estimator so we want to find a model with the best score distribution for a given set of hyperparameters
 - (note: if we want to do hyperparaemter tuning and want our model selection to be unbiaesed weshould do nested Cross Validaiton, but for simpliciy of the code we are sticking with kfold. See: xxxxx_xxxx_xx.ipynb for the nested cross validaiton example)